# Day3_0 별첨: 데이터클래스 (Dataclass)

## 📚 학습 목표

1. **데이터클래스의 필요성** 이해하기
2. **`@dataclass` 데코레이터** 활용하기
3. **Field 함수**로 고급 기능 사용하기
4. **비교와 정렬** 자동화하기
5. **불변 데이터클래스** (frozen) 만들기
6. **실무 활용 패턴** 마스터하기

---

## 🎯 왜 데이터클래스를 배우나요?

| 문제 | 전통적 방법 | 데이터클래스 해결 |
|------|-----------|------------------|
| 반복적인 `__init__` 작성 | 매번 직접 작성 | 자동 생성 |
| `__repr__` 구현 | 디버깅용 출력 직접 작성 | 자동 생성 |
| `__eq__` 구현 | 비교 로직 직접 작성 | 자동 생성 |
| 불변 객체 만들기 | property + 검증 코드 | `frozen=True` 한 줄 |
| 기본값 설정 | 복잡한 패턴 필요 | `field(default_factory=...)` |

**분석가 관점**: 데이터 분석 결과, 설정값, API 응답 등을 깔끔하게 구조화할 수 있습니다!

---

## 1. 데이터클래스의 필요성

### 전통적인 클래스의 문제점

데이터를 저장하는 간단한 클래스를 만들 때, 반복적인 코드를 많이 작성해야 합니다.

In [ ]:
# ❌ 전통적인 방법: 반복 코드가 많음
class StudentOld:
    """학생 정보 클래스 (전통적 방법)"""
    
    def __init__(self, name, age, grade, major):
        self.name = name
        self.age = age
        self.grade = grade
        self.major = major
    
    def __repr__(self):
        return f"StudentOld(name='{self.name}', age={self.age}, grade={self.grade}, major='{self.major}')"
    
    def __eq__(self, other):
        if not isinstance(other, StudentOld):
            return False
        return (self.name == other.name and 
                self.age == other.age and 
                self.grade == other.grade and 
                self.major == other.major)

# 사용
student1 = StudentOld("김철수", 20, 3.8, "컴퓨터공학")
student2 = StudentOld("김철수", 20, 3.8, "컴퓨터공학")

print(student1)
print(f"같은 학생? {student1 == student2}")

In [ ]:
# ✅ 데이터클래스: 간결하고 명확
from dataclasses import dataclass

@dataclass
class Student:
    """학생 정보 클래스 (데이터클래스)"""
    name: str
    age: int
    grade: float
    major: str

# 사용 - 동일한 기능을 자동으로 제공!
student1 = Student("김철수", 20, 3.8, "컴퓨터공학")
student2 = Student("김철수", 20, 3.8, "컴퓨터공학")

print(student1)  # __repr__ 자동 생성
print(f"같은 학생? {student1 == student2}")  # __eq__ 자동 생성

### 💡 코드 비교

| 항목 | 전통적 클래스 | 데이터클래스 |
|------|-------------|-------------|
| 코드 라인 수 | 20줄 | 7줄 |
| `__init__` | 직접 작성 | 자동 생성 |
| `__repr__` | 직접 작성 | 자동 생성 |
| `__eq__` | 직접 작성 | 자동 생성 |
| 타입 힌트 | 선택 | 필수 (문서화 효과) |

**결론**: 데이터클래스는 **보일러플레이트 코드를 70% 이상 줄여줍니다!**

---

## 2. 데이터클래스 기본 사용법

### 2.1 기본 구조

In [ ]:
from dataclasses import dataclass

@dataclass
class Product:
    """제품 정보"""
    name: str
    price: int
    stock: int
    category: str

# 인스턴스 생성
laptop = Product("맥북 프로", 2500000, 10, "전자기기")
mouse = Product("로지텍 MX", 89000, 50, "전자기기")

print(laptop)
print(f"\n가격: {laptop.price:,}원")
print(f"재고: {laptop.stock}개")

### 2.2 기본값 설정

In [ ]:
@dataclass
class User:
    """사용자 정보"""
    username: str
    email: str
    is_active: bool = True  # 기본값
    role: str = "user"      # 기본값

# 기본값 활용
user1 = User("dante", "dante@example.com")
user2 = User("admin", "admin@example.com", is_active=True, role="admin")

print(user1)
print(user2)
print(f"\nuser1 활성화? {user1.is_active}")
print(f"user1 역할: {user1.role}")

### ⚠️ 주의: 가변 객체 기본값

**절대로 리스트나 딕셔너리를 직접 기본값으로 사용하지 마세요!**

In [ ]:
# ❌ 잘못된 방법 - 에러 발생!
# @dataclass
# class TaskListBad:
#     tasks: list = []  # ValueError: mutable default 사용 불가!

# ✅ 올바른 방법: field(default_factory=...) 사용
from dataclasses import dataclass, field

@dataclass
class TaskList:
    """할 일 목록"""
    owner: str
    tasks: list = field(default_factory=list)  # 각 인스턴스마다 새 리스트 생성
    metadata: dict = field(default_factory=dict)

# 각 인스턴스가 독립적인 리스트를 가짐
list1 = TaskList("김철수")
list2 = TaskList("이영희")

list1.tasks.append("과제 제출")
list2.tasks.append("운동하기")

print(f"list1: {list1}")
print(f"list2: {list2}")
print(f"\n독립적? {list1.tasks is not list2.tasks}")  # True

---

## 3. Field 함수 고급 기능

### 3.1 field() 매개변수

| 매개변수 | 설명 | 예시 |
|---------|------|------|
| `default` | 기본값 | `field(default=0)` |
| `default_factory` | 기본값 생성 함수 | `field(default_factory=list)` |
| `init` | `__init__`에 포함 여부 | `field(init=False)` |
| `repr` | `__repr__`에 포함 여부 | `field(repr=False)` |
| `compare` | 비교에 포함 여부 | `field(compare=False)` |
| `metadata` | 메타데이터 저장 | `field(metadata={'unit': 'cm'})` |

In [ ]:
from dataclasses import dataclass, field
from datetime import datetime

@dataclass
class BlogPost:
    """블로그 포스트"""
    title: str
    content: str
    author: str
    
    # init=False: __init__에서 받지 않음 (자동 생성)
    created_at: datetime = field(init=False, default_factory=datetime.now)
    
    # repr=False: 출력에서 제외 (민감한 정보)
    internal_id: str = field(default="", repr=False)
    
    # compare=False: 비교에서 제외
    view_count: int = field(default=0, compare=False)
    
    # metadata: 추가 정보 저장
    tags: list = field(default_factory=list, metadata={'description': '태그 목록'})

# 사용
post1 = BlogPost("Python 데이터클래스", "데이터클래스는 편리합니다.", "단테")
post2 = BlogPost("Python 데이터클래스", "데이터클래스는 편리합니다.", "단테")

print(post1)  # internal_id는 출력 안 됨 (repr=False)
print(f"\n생성 시각: {post1.created_at}")

# view_count가 달라도 같은 포스트로 인식 (compare=False)
post2.view_count = 100
print(f"\n같은 포스트? {post1 == post2}")  # True

### 3.2 계산된 필드 (Computed Field)

In [ ]:
@dataclass
class Rectangle:
    """직사각형"""
    width: float
    height: float
    
    # init=False: __init__에서 받지 않고, __post_init__에서 계산
    area: float = field(init=False)
    perimeter: float = field(init=False)
    
    def __post_init__(self):
        """인스턴스 생성 후 자동 호출"""
        self.area = self.width * self.height
        self.perimeter = 2 * (self.width + self.height)

# 사용
rect = Rectangle(10, 5)
print(rect)
print(f"넓이: {rect.area}")
print(f"둘레: {rect.perimeter}")

---

## 4. 메서드와 프로퍼티 추가

데이터클래스에도 일반 클래스처럼 메서드를 추가할 수 있습니다.

In [ ]:
@dataclass
class BankAccount:
    """은행 계좌"""
    owner: str
    account_number: str
    balance: float = 0.0
    transactions: list = field(default_factory=list, repr=False)
    
    def deposit(self, amount: float) -> str:
        """입금"""
        if amount <= 0:
            return "입금액은 0보다 커야 합니다."
        
        self.balance += amount
        self.transactions.append(f"입금: +{amount:,}원")
        return f"{amount:,}원 입금 완료. 잔액: {self.balance:,}원"
    
    def withdraw(self, amount: float) -> str:
        """출금"""
        if amount <= 0:
            return "출금액은 0보다 커야 합니다."
        if amount > self.balance:
            return f"잔액 부족! 현재 잔액: {self.balance:,}원"
        
        self.balance -= amount
        self.transactions.append(f"출금: -{amount:,}원")
        return f"{amount:,}원 출금 완료. 잔액: {self.balance:,}원"
    
    @property
    def transaction_history(self) -> str:
        """거래 내역"""
        if not self.transactions:
            return "거래 내역 없음"
        return "\n".join(self.transactions)

# 사용
account = BankAccount("김철수", "110-123-456789", 100000)
print(account)

print(f"\n{account.deposit(50000)}")
print(account.withdraw(30000))
print(account.withdraw(200000))

print(f"\n=== 거래 내역 ===")
print(account.transaction_history)

---

## 5. 비교와 정렬: order=True

`order=True`를 설정하면 `<`, `<=`, `>`, `>=` 비교 연산자가 자동으로 생성됩니다.

In [ ]:
from dataclasses import dataclass

@dataclass(order=True)
class Student:
    """학생 (성적순 정렬 가능)"""
    name: str
    score: float
    
    # order=True는 필드 순서대로 비교 (name → score)
    # score로만 비교하려면 sort_index 사용

# 학생 목록
students = [
    Student("김철수", 85.5),
    Student("이영희", 92.0),
    Student("박민수", 78.0),
    Student("최지우", 88.5),
]

# 비교 연산
print(f"이영희 > 김철수? {students[1] > students[0]}")

# 정렬 (name 기준 → score 기준)
print("\n=== 기본 정렬 (이름순) ===")
for s in sorted(students):
    print(s)

### 💡 특정 필드로만 정렬하기

In [ ]:
from dataclasses import dataclass, field

@dataclass(order=True)
class Student:
    """학생 (성적순 정렬)"""
    # sort_index: 정렬에만 사용되는 필드 (init=False)
    sort_index: float = field(init=False, repr=False)
    
    name: str
    score: float
    
    def __post_init__(self):
        # 정렬 기준: 성적 (내림차순을 위해 음수)
        self.sort_index = -self.score

# 학생 목록
students = [
    Student("김철수", 85.5),
    Student("이영희", 92.0),
    Student("박민수", 78.0),
    Student("최지우", 88.5),
]

# 성적순 정렬 (내림차순)
print("=== 성적순 정렬 (높은 순) ===")
for s in sorted(students):
    print(f"{s.name}: {s.score}점")

---

## 6. 불변 데이터클래스: frozen=True

`frozen=True`로 설정하면 **인스턴스 생성 후 값을 변경할 수 없습니다**.  
튜플처럼 불변 객체가 됩니다.

In [ ]:
@dataclass(frozen=True)
class Point:
    """2D 좌표 (불변)"""
    x: float
    y: float
    
    def distance_from_origin(self) -> float:
        """원점으로부터의 거리"""
        return (self.x ** 2 + self.y ** 2) ** 0.5

# 생성
p1 = Point(3, 4)
print(p1)
print(f"원점 거리: {p1.distance_from_origin()}")

# 수정 불가!
try:
    p1.x = 10  # FrozenInstanceError
except Exception as e:
    print(f"\n에러: {type(e).__name__}")
    print("frozen=True로 설정되어 값을 변경할 수 없습니다!")

### 💡 frozen=True의 장점

1. **딕셔너리 키로 사용 가능** (해시 가능)
2. **스레드 안전** (동시성 환경에서 안전)
3. **의도하지 않은 변경 방지** (버그 예방)

In [ ]:
@dataclass(frozen=True)
class Coordinate:
    """좌표 (불변)"""
    lat: float  # 위도
    lon: float  # 경도

# frozen=True이므로 딕셔너리 키로 사용 가능
locations = {
    Coordinate(37.5665, 126.9780): "서울시청",
    Coordinate(35.1796, 129.0756): "부산역",
    Coordinate(37.4563, 126.7052): "인천공항",
}

# 조회
seoul = Coordinate(37.5665, 126.9780)
print(f"서울시청 좌표: {locations[seoul]}")

# frozen이 아니면 딕셔너리 키로 사용 불가
# @dataclass
# class MutableCoord:
#     lat: float
#     lon: float
# 
# d = {MutableCoord(1, 2): "test"}  # TypeError: unhashable type

---

## 7. 상속

데이터클래스도 일반 클래스처럼 상속할 수 있습니다.

In [ ]:
@dataclass
class Person:
    """사람 (기본 클래스)"""
    name: str
    age: int
    
    def introduce(self) -> str:
        return f"안녕하세요, 저는 {self.name}이고 {self.age}살입니다."

@dataclass
class Employee(Person):
    """직원 (Person 상속)"""
    employee_id: str
    department: str
    salary: int
    
    def introduce(self) -> str:
        base = super().introduce()
        return f"{base} {self.department} 부서에서 근무합니다."

# 사용
person = Person("김철수", 25)
employee = Employee("이영희", 30, "E001", "개발팀", 5000000)

print(person.introduce())
print(employee.introduce())
print(f"\n급여: {employee.salary:,}원")

---

## 💡 실무 예시 1: 데이터 분석 결과

데이터 분석 결과를 구조화하여 저장하고 관리합니다.

In [ ]:
from dataclasses import dataclass, field
from datetime import datetime
from typing import List, Dict

@dataclass
class AnalysisResult:
    """데이터 분석 결과"""
    dataset_name: str
    analyst: str
    row_count: int
    column_count: int
    
    # 통계 정보
    statistics: Dict[str, float] = field(default_factory=dict)
    
    # 인사이트
    insights: List[str] = field(default_factory=list)
    
    # 메타데이터
    created_at: datetime = field(default_factory=datetime.now, repr=False)
    
    def add_insight(self, insight: str) -> None:
        """인사이트 추가"""
        self.insights.append(insight)
    
    def summary(self) -> str:
        """분석 요약"""
        return f"""
=== 분석 결과 요약 ===
데이터셋: {self.dataset_name}
분석자: {self.analyst}
데이터 크기: {self.row_count:,}행 x {self.column_count}열
주요 통계: {self.statistics}
인사이트 개수: {len(self.insights)}개
분석 시각: {self.created_at.strftime('%Y-%m-%d %H:%M:%S')}
"""

# 사용
result = AnalysisResult(
    dataset_name="sales_2024.csv",
    analyst="단테",
    row_count=10000,
    column_count=15
)

result.statistics = {
    "mean_sales": 1250000,
    "median_sales": 980000,
    "std_sales": 350000
}

result.add_insight("상위 10% 제품이 전체 매출의 60% 차지")
result.add_insight("계절별 매출 패턴 뚜렷함 (여름 성수기)")
result.add_insight("모바일 구매 비율 전년 대비 25% 증가")

print(result.summary())

print("=== 주요 인사이트 ===")
for i, insight in enumerate(result.insights, 1):
    print(f"{i}. {insight}")

---

## 💡 실무 예시 2: API 응답 구조화

In [ ]:
from dataclasses import dataclass, asdict
from typing import Optional
import json

@dataclass
class APIResponse:
    """API 응답 표준 구조"""
    success: bool
    message: str
    data: Optional[dict] = None
    error_code: Optional[str] = None
    
    def to_json(self) -> str:
        """JSON으로 변환"""
        return json.dumps(asdict(self), ensure_ascii=False, indent=2)

@dataclass
class UserData:
    """사용자 데이터"""
    user_id: int
    username: str
    email: str
    is_premium: bool = False

# 성공 응답
user = UserData(1001, "dante", "dante@example.com", True)
success_response = APIResponse(
    success=True,
    message="사용자 정보 조회 성공",
    data=asdict(user)
)

print("=== 성공 응답 ===")
print(success_response.to_json())

# 실패 응답
error_response = APIResponse(
    success=False,
    message="사용자를 찾을 수 없습니다",
    error_code="USER_NOT_FOUND"
)

print("\n=== 실패 응답 ===")
print(error_response.to_json())

---

## 💡 실무 예시 3: 설정 관리

In [ ]:
from dataclasses import dataclass, field
from typing import List

@dataclass(frozen=True)
class DatabaseConfig:
    """데이터베이스 설정 (불변)"""
    host: str = "localhost"
    port: int = 5432
    database: str = "mydb"
    user: str = "admin"
    password: str = field(repr=False, default="password123")  # 출력 시 숨김
    
    @property
    def connection_string(self) -> str:
        """연결 문자열"""
        return f"postgresql://{self.user}:***@{self.host}:{self.port}/{self.database}"

@dataclass
class AppConfig:
    """애플리케이션 설정"""
    app_name: str
    version: str
    debug: bool = False
    
    # 데이터베이스 설정
    database: DatabaseConfig = field(default_factory=DatabaseConfig)
    
    # 허용 호스트
    allowed_hosts: List[str] = field(default_factory=lambda: ["localhost", "127.0.0.1"])
    
    def validate(self) -> bool:
        """설정 검증"""
        if not self.app_name:
            raise ValueError("app_name은 필수입니다")
        if not self.allowed_hosts:
            raise ValueError("allowed_hosts는 비어있을 수 없습니다")
        return True

# 개발 환경 설정
dev_config = AppConfig(
    app_name="MyApp",
    version="1.0.0",
    debug=True
)

print("=== 개발 환경 설정 ===")
print(dev_config)
print(f"\nDB 연결: {dev_config.database.connection_string}")
print(f"디버그 모드: {dev_config.debug}")
print(f"허용 호스트: {dev_config.allowed_hosts}")

# 프로덕션 환경 설정
prod_db = DatabaseConfig(
    host="prod.example.com",
    database="prod_db",
    user="prod_user",
    password="secure_password"
)

prod_config = AppConfig(
    app_name="MyApp",
    version="1.0.0",
    debug=False,
    database=prod_db,
    allowed_hosts=["myapp.com", "www.myapp.com"]
)

print("\n=== 프로덕션 환경 설정 ===")
print(prod_config)
print(f"\nDB 연결: {prod_config.database.connection_string}")

# 설정 검증
try:
    prod_config.validate()
    print("\n✅ 설정 검증 통과")
except ValueError as e:
    print(f"\n❌ 설정 오류: {e}")

---

## 🎯 실습 퀴즈

---

### Q1. 기본 데이터클래스 생성 ⭐

**문제**: `Book` 데이터클래스를 만드세요.

**요구사항**:
- 필드: `title` (제목), `author` (저자), `price` (가격), `isbn` (ISBN)
- 모든 필드는 문자열 타입
- `price`만 정수 타입

In [ ]:
# 여기에 코드를 작성하세요
from dataclasses import dataclass

# Book 데이터클래스 작성

# 테스트
book = Book("파이썬 데이터 분석", "단테", 35000, "978-89-xxxxx")
print(book)
print(f"가격: {book.price:,}원")

### Q2. 기본값과 default_factory ⭐⭐

**문제**: `ShoppingCart` 데이터클래스를 만드세요.

**요구사항**:
- 필드: `user_id` (사용자 ID, 문자열)
- 필드: `items` (상품 목록, 리스트) - 기본값은 빈 리스트
- 필드: `is_active` (활성 여부, bool) - 기본값 `True`
- 메서드: `add_item(item_name)` - 상품 추가
- 메서드: `total_items()` - 총 상품 개수 반환

In [ ]:
# 여기에 코드를 작성하세요
from dataclasses import dataclass, field

# ShoppingCart 작성

# 테스트
cart1 = ShoppingCart("user001")
cart1.add_item("노트북")
cart1.add_item("마우스")

cart2 = ShoppingCart("user002")
cart2.add_item("키보드")

print(f"cart1: {cart1.items}, 총 {cart1.total_items()}개")
print(f"cart2: {cart2.items}, 총 {cart2.total_items()}개")
print(f"독립적? {cart1.items is not cart2.items}")

### Q3. __post_init__으로 계산된 필드 ⭐⭐⭐

**문제**: `Circle` 데이터클래스를 만드세요.

**요구사항**:
- 필드: `radius` (반지름, float)
- 계산 필드: `area` (넓이) - `__post_init__`에서 계산 (π * r²)
- 계산 필드: `circumference` (둘레) - `__post_init__`에서 계산 (2 * π * r)
- `area`와 `circumference`는 `init=False`로 설정
- π는 3.14159 사용

In [ ]:
# 여기에 코드를 작성하세요
from dataclasses import dataclass, field

# Circle 작성

# 테스트
circle = Circle(5.0)
print(circle)
print(f"넓이: {circle.area:.2f}")
print(f"둘레: {circle.circumference:.2f}")

### Q4. order=True로 정렬 ⭐⭐⭐

**문제**: `Product` 데이터클래스를 가격순으로 정렬 가능하게 만드세요.

**요구사항**:
- 필드: `name` (제품명, str), `price` (가격, int), `stock` (재고, int)
- `order=True` 설정
- `sort_index` 필드 추가 (가격순 정렬용, init=False, repr=False)
- `__post_init__`에서 `sort_index = self.price` 설정
- 리스트를 정렬하면 가격 낮은 순으로 정렬되어야 함

In [ ]:
# 여기에 코드를 작성하세요
from dataclasses import dataclass, field

# Product 작성

# 테스트
products = [
    Product("노트북", 1500000, 10),
    Product("마우스", 25000, 50),
    Product("키보드", 89000, 30),
    Product("모니터", 350000, 15),
]

print("=== 가격순 정렬 ===")
for p in sorted(products):
    print(f"{p.name}: {p.price:,}원")

### Q5. frozen=True 불변 객체 ⭐⭐⭐⭐

**문제**: `Config` 데이터클래스를 불변으로 만들고 딕셔너리 키로 사용하세요.

**요구사항**:
- 필드: `env` (환경, str), `debug` (디버그 모드, bool)
- `frozen=True` 설정
- 생성 후 값 변경 시도하면 에러 발생
- 딕셔너리 키로 사용 가능 (해시 가능)

In [ ]:
# 여기에 코드를 작성하세요
from dataclasses import dataclass

# Config 작성

# 테스트
dev_config = Config("development", True)
prod_config = Config("production", False)

# 딕셔너리 키로 사용
settings = {
    dev_config: {"db_host": "localhost", "cache": False},
    prod_config: {"db_host": "prod.db.com", "cache": True},
}

print(f"개발 환경 설정: {settings[dev_config]}")
print(f"프로덕션 환경 설정: {settings[prod_config]}")

# 수정 시도 (에러 발생 확인)
try:
    dev_config.debug = False
except Exception as e:
    print(f"\n예상된 에러: {type(e).__name__}")

### Q6. 종합 실무 예제 ⭐⭐⭐⭐⭐

**문제**: 로그 분석기 데이터클래스를 만드세요.

**요구사항**:

**LogEntry 데이터클래스**:
- 필드: `timestamp` (str), `level` (str), `message` (str)
- `level`은 "INFO", "WARNING", "ERROR" 중 하나

**LogAnalyzer 데이터클래스**:
- 필드: `source` (소스명, str)
- 필드: `logs` (로그 목록, List[LogEntry]) - default_factory=list
- 필드: `stats` (통계, dict) - init=False
- 메서드: `add_log(timestamp, level, message)` - 로그 추가
- 메서드: `analyze()` - 로그 레벨별 개수를 stats에 저장
- 메서드: `get_errors()` - ERROR 로그만 반환
- `__post_init__`에서 `self.stats = {}` 초기화

In [ ]:
# 여기에 코드를 작성하세요
from dataclasses import dataclass, field
from typing import List

# LogEntry 작성

# LogAnalyzer 작성

# 테스트
analyzer = LogAnalyzer("app.log")

analyzer.add_log("2024-01-15 10:00:00", "INFO", "서버 시작")
analyzer.add_log("2024-01-15 10:05:30", "INFO", "사용자 로그인")
analyzer.add_log("2024-01-15 10:10:15", "WARNING", "메모리 사용량 80% 초과")
analyzer.add_log("2024-01-15 10:15:00", "ERROR", "데이터베이스 연결 실패")
analyzer.add_log("2024-01-15 10:20:00", "ERROR", "API 요청 타임아웃")
analyzer.add_log("2024-01-15 10:25:00", "INFO", "정상 복구")

# 분석
analyzer.analyze()
print(f"총 로그 수: {len(analyzer.logs)}개")
print(f"통계: {analyzer.stats}")

# 에러 로그만 출력
print("\n=== 에러 로그 ===")
for log in analyzer.get_errors():
    print(f"[{log.timestamp}] {log.message}")

---

## 📊 학습 정리

### 핵심 개념 복습

| 개념 | 문법 | 사용 시기 |
|------|------|----------|
| 기본 데이터클래스 | `@dataclass` | 데이터 저장용 클래스 |
| 기본값 | `field(default=...)` | 불변 기본값 |
| 가변 기본값 | `field(default_factory=...)` | 리스트, 딕셔너리 |
| 계산 필드 | `field(init=False)` + `__post_init__` | 다른 필드로부터 계산 |
| 정렬 | `@dataclass(order=True)` | 객체 비교/정렬 |
| 불변 객체 | `@dataclass(frozen=True)` | 변경 불가능한 객체 |
| 필드 제외 | `field(repr=False)` | 출력에서 숨김 |
| 비교 제외 | `field(compare=False)` | 비교에서 제외 |

### 데이터클래스 vs 일반 클래스

| 특징 | 일반 클래스 | 데이터클래스 |
|------|-----------|-------------|
| `__init__` | 직접 작성 | 자동 생성 |
| `__repr__` | 직접 작성 | 자동 생성 |
| `__eq__` | 직접 작성 | 자동 생성 |
| 타입 힌트 | 선택 | 필수 (권장) |
| 코드량 | 많음 | 적음 (70% 감소) |
| 가독성 | 보통 | 매우 높음 |

### 💡 실무 활용 패턴

1. **데이터 전송 객체 (DTO)**: API 요청/응답 구조화
2. **설정 관리**: 애플리케이션 설정값 관리 (frozen=True)
3. **분석 결과**: 데이터 분석 결과 저장 및 공유
4. **도메인 모델**: 비즈니스 로직의 핵심 객체
5. **불변 객체**: 좌표, 날짜 등 변경되지 않아야 하는 값

### 언제 데이터클래스를 사용할까?

**✅ 사용하기 좋은 경우**:
- 주로 데이터를 저장하는 클래스
- 여러 필드를 가진 구조화된 데이터
- `__init__`, `__repr__`, `__eq__`가 필요한 경우
- 타입 힌트로 명확한 문서화가 필요한 경우

**❌ 사용하지 않는 것이 좋은 경우**:
- 복잡한 상속 구조
- 동적으로 속성이 추가/삭제되는 경우
- 메서드 중심의 클래스 (데이터보다 동작이 중요)

---

## 🎓 다음 단계

데이터클래스를 마스터했다면:
- **Pydantic**: 데이터 검증 강화 라이브러리
- **attrs**: 데이터클래스의 대안 (더 많은 기능)
- **msgspec**: 고성능 직렬화/역직렬화

**수고하셨습니다!** 🎉